In [1]:
import pandas as pd
import numpy as np


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet


!pip install lightgbm
from lightgbm import LGBMRegressor
from lightgbm import plot_importance


# Model Tuning

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score


In [2]:
agri_data=pd.read_excel('FPData v1.2.xlsx')
agri_data.head()

,Date,COMM_NAME,COMM_CODE,COMM_CATEGORY,COMM_WT,Monthly Price,Rainfall
0,2011-04-01,Paddy,1101010101,CEREALS,1.43052,97.5,41.00
1,2011-04-01,Paddy,1101010101,CEREALS,1.43052,97.5,40.59
2,2011-04-01,Wheat,1101010102,CEREALS,1.02823,101.1,41.00
3,2011-04-01,Wheat,1101010102,CEREALS,1.02823,101.1,40.59
4,2011-04-01,Jowar,1101010103,CEREALS,0.06764,84.5,41.00


In [3]:
agri_data.dtypes


Date             datetime64[ns]
COMM_NAME                object
COMM_CODE                 int64
COMM_CATEGORY            object
COMM_WT                 float64
Monthly Price           float64
Rainfall                float64
dtype: object

In [4]:
# extract month and year from dates
agri_data['Month'] = [i.month for i in agri_data['Date']]
agri_data['year'] = [i.year for i in agri_data['Date']]
#agri_data['flag'] = pd.Series(np.where(agri_data['Date'] >= np.datetime64('2020-03-03'), 1, 0)) #flag for COVID-19
agri_data.head()

,Date,COMM_NAME,COMM_CODE,COMM_CATEGORY,COMM_WT,Monthly Price,Rainfall,Month,year
0,2011-04-01,Paddy,1101010101,CEREALS,1.43052,97.5,41.00,4,2011
1,2011-04-01,Paddy,1101010101,CEREALS,1.43052,97.5,40.59,4,2011
2,2011-04-01,Wheat,1101010102,CEREALS,1.02823,101.1,41.00,4,2011
3,2011-04-01,Wheat,1101010102,CEREALS,1.02823,101.1,40.59,4,2011
4,2011-04-01,Jowar,1101010103,CEREALS,0.06764,84.5,41.00,4,2011


In [5]:
agri_data.columns

Index(['Date', 'COMM_NAME', 'COMM_CODE', 'COMM_CATEGORY', 'COMM_WT',
       'Monthly Price', 'Rainfall', 'Month', 'year'],
      dtype='object')

In [6]:
target='Monthly Price'

In [7]:
agri_data.set_index('Date', inplace=True)
agri_data.head()

,COMM_NAME,COMM_CODE,COMM_CATEGORY,COMM_WT,Monthly Price,Rainfall,Month,year
Date,,,,,,,,
2011-04-01,Paddy,1101010101,CEREALS,1.43052,97.5,41.00,4,2011
2011-04-01,Paddy,1101010101,CEREALS,1.43052,97.5,40.59,4,2011
2011-04-01,Wheat,1101010102,CEREALS,1.02823,101.1,41.00,4,2011
2011-04-01,Wheat,1101010102,CEREALS,1.02823,101.1,40.59,4,2011
2011-04-01,Jowar,1101010103,CEREALS,0.06764,84.5,41.00,4,2011


In [66]:
# Filtering out records in the dataframe with NULLs in Monthly Price
unknown_price_df=agri_data[agri_data['Monthly Price'].isna()]
# Filtering out records for June, 2021 to create a forecast dataframe
forecast_agri_df=unknown_price_df[(unknown_price_df['year']>2021) |((unknown_price_df['year']==2021) & (unknown_price_df['Month']>6))]
forecast_agri_index=forecast_agri_df.index  #indexes of the records in forecast dataframe
#Eliminating the records that need to be appear in forecast data from training data
agri_data=agri_data.drop(index=forecast_agri_index)

In [79]:
# split data into train-test set
train = agri_data[agri_data['year'] < 2020]
test = agri_data[agri_data['year'] >= 2020]
#check shape
train.shape, test.shape

((11016, 8), (1224, 8))

In [80]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
train['COMM_NAME'] = lbl.fit_transform(train['COMM_NAME'].astype(str))
train['COMM_CATEGORY'] = lbl.fit_transform(train['COMM_CATEGORY'].astype(str))
test['COMM_NAME'] = lbl.fit_transform(test['COMM_NAME'].astype(str))
test['COMM_CATEGORY'] = lbl.fit_transform(test['COMM_CATEGORY'].astype(str))

C:\Users\Dillu\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Dillu\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Dillu\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [81]:
train.dtypes

COMM_NAME          int32
COMM_CODE          int64
COMM_CATEGORY      int32
COMM_WT          float64
Monthly Price    float64
Rainfall         float64
Month              int64
year               int64
dtype: object

In [82]:
train=train.dropna()
test=test.dropna()
train.shape, test.shape

((10524, 8), (780, 8))

In [83]:
X_train=train.drop('Monthly Price',axis=1)
y_train=train['Monthly Price']
X_test=test.drop('Monthly Price',axis=1)
y_test=test['Monthly Price']

In [84]:
train.dtypes

COMM_NAME          int32
COMM_CODE          int64
COMM_CATEGORY      int32
COMM_WT          float64
Monthly Price    float64
Rainfall         float64
Month              int64
year               int64
dtype: object

In [85]:
regressor=LGBMRegressor(bagging_fraction=0.7, bagging_freq=6, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
              importance_type='split', learning_rate=0.4, max_depth=-1,
              min_child_samples=1, min_child_weight=0.001, min_split_gain=0.2,
              n_estimators=100, n_jobs=-1, num_leaves=8, objective=None,
              random_state=1212, reg_alpha=0.1, reg_lambda=0.3, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [86]:
lgb_model = regressor.fit(X_train, y_train)
y_pred = lgb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


40.30845519866382

In [87]:
y_pred = lgb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

40.30845519866382